In [1]:
import pandas as pd

In [2]:
result=pd.read_excel("/home/d4k/Documents/guillermo/doctorado/systematic review/ouput_excel/result_WOW_ACM_DBLP.xlsx")

In [7]:
rows_with_blank_abstracts = result[result["DOI"].isnull() | (result["DOI"] == '')| (result["DOI"] == ' ')]

In [8]:
rows_with_blank_abstracts

,Authors,Researcher Ids,ORCIDs,title,Volume,Issue,DOI,Document Type,Publication Date,Abstract,ISSN,eISSN,ISBN,Pages,Publisher,Proceedings title,Keywords
0,"Xenos, Alexandros; Pavlopoulos, John; Androuts...","Pavlopoulos, John/GPP-2913-2022","Pavlopoulos, John/0000-0001-9188-7425; Androut...",Context Sensitivity Estimation in Toxicity Det...,NaN,NaN,,Proceedings Paper,2021.0,User posts whose perceived toxicity depends on...,NaN,NaN,978-1-954085-59-6,140-145,,5th Workshop on Structured Prediction for NLP...,NaN
3,"Brassard-Gourdeau, Eloi; Khoury, Richard",NaN,NaN,Subversive Toxicity Detection using Sentiment ...,NaN,NaN,,Proceedings Paper,2019.0,The presence of toxic content has become a maj...,NaN,NaN,978-1-950737-43-7,1-10,,3rd Workshop on Abusive Language Online3rd Wo...,NaN
4,"Fortuna, Paula; Soler-Company, Juan; Wanner, Leo",NaN,NaN,"Toxic, Hateful, Offensive or Abusive? What Are...",NaN,NaN,,Proceedings Paper,2020.0,The field of the automatic detection of hate s...,NaN,NaN,979-10-95546-34-4,6786-6794,,12th International Conference on Language Res...,NaN
10,"Pavlopoulos, John; Sorensen, Jeffrey; Dixon, L...","Pavlopoulos, John/GPP-2913-2022; Dixon, Lucas/...","Pavlopoulos, John/0000-0001-9188-7425; Androut...",Toxicity Detection: Does Context Really Matter?,NaN,NaN,,Proceedings Paper,2020.0,Moderation is crucial to promoting healthy onl...,NaN,NaN,978-1-952148-25-5,4296-4305,,58th Annual Meeting of the Association-for-Co...,NaN
15,"Sap, Maarten; Swayamdipta, Swabha; Vianna, Lau...",NaN,NaN,Annotators with Attitudes: How Annotator Belie...,NaN,NaN,,Proceedings Paper,2022.0,The perceived toxicity of language can vary ba...,NaN,NaN,978-1-955917-71-1,5884-5906,,Conference of the North-American-Chapter-of-t...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,"Rodriguez N,Galeano SR",NaN,NaN,Shielding Google's language toxicity model aga...,abs/1801.01828,NaN,,Journal Article,2018.0,NaN,NaN,NaN,NaN,NaN,CoRR,,NaN
451,"Miller B,van den Bosch A,Kunzelman C,Olive J,S...",NaN,NaN,Notoriously Toxic: The Language and Cost of Ha...,NaN,NaN,,Conference Paper,2016.0,NaN,NaN,NaN,NaN,NaN,Alliance of Digital Humanities Organizations (...,11th Annual International Conference of the Al...,NaN
453,"Morgan S,Ranasinghe T,Zampieri M",NaN,NaN,WLV-RIT at GermEval 2021: Multitask Learning w...,NaN,NaN,,Conference Paper,2021.0,NaN,NaN,NaN,NaN,NaN,Association for Computational Linguistics,Proceedings of the GermEval 2021 Shared Task o...,NaN
455,"de Paula AF,Schlicht IB",NaN,NaN,AI-UPV at IberLEF-2021 DETOXIS task: Toxicity ...,2943,NaN,,Conference Paper,2021.0,NaN,NaN,NaN,NaN,NaN,CEUR-WS.org,Proceedings of the Iberian Languages Evaluatio...,NaN


In [58]:
from googlesearch import search
import requests
import pdfkit
import pdfplumber
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from habanero import Crossref
from crossref.restful import Works
from bs4 import BeautifulSoup
import random 

from time import sleep
import pandas as pd

class AbstractExtractor:
    def __init__(self,dataframe) -> None:
        self.dataframe=dataframe
        self.llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral", gpu_layers=100,max_new_tokens = 4096,
                                           context_length = 4096)
        self.autotokenizer= AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
        self.cr = Crossref()
        self.works = Works()
        print("Initializacion finished")
    def search_link(self,query,num_results=10,proxy=None,sleep_interval=0):
        links=search(query,num_results=num_results,proxy=proxy,sleep_interval=sleep_interval)
        return links
    
    def get_DOI(self,query):
        try:
            result = self.cr.works(query=query)
            if 'items' in result['message'] and result['message']['items']:
                
                return result['message']['items'][0]['DOI']
            else:
               
                return None
        except Exception as e:
            print(f"Error obtaining DOI in query:'{query}': {e}")
            
            return None
    def populate_DOI(self):
        rows_with_blank_abstracts = result[result["DOI"].isnull() | (result["DOI"] == '')| (result["DOI"] == ' ')]
        for index, row in rows_with_blank_abstracts.iterrows():
            doi=self.get_DOI(row["title"])
            if doi is not None:
                self.dataframe.at[index,"DOI"]=doi
            sleep(random.uniform(0.1,3.0))

    def abstract_from_DOI(self,DOI):
        result=self.works.doi(DOI)
        try:
            abstract = result['abstract']
            return abstract
        except KeyError:
            return None
    def extractabstract(self,pdf_name="out.pdf"):
        with pdfplumber.open(pdf_name) as pdf:
                pages = pdf.pages
                for page in pages:
                    text = page.extract_text()
                    for chunk in self.chunk_text(text):
                        instruction = f"""
                            <s>[INST] You are an expert reader assistant. Your task is to extract the abstract of a research from the given text, and you must provide the abstract exactly as it is formatted. If the provided text does not contain an abstract, the output format must be "No abstract provided."
                            Text: {chunk}
                            Abstract:[/INST]</s>
                            """
                        output=self.llm(instruction,temperature=1.0,seed=42)
                        if not "No abstract provided" in output:
                            return output
                            break
                return None
    def download_html(self,url,save_path):
        try:
            # Send a GET request to the URL
            session = requests.Session()
            response = session.get(url)
            # response = requests.get(url,headers=headers)
            response.raise_for_status()  # Raise an exception for HTTP errors
            with open(save_path, 'w', encoding='utf-8') as file:
                file.write(response.text)
            return response.text
        
        except requests.exceptions.HTTPError as errh:
            print(f"HTTP Error: {errh}")
        except requests.exceptions.ConnectionError as errc:
            print(f"Error Connecting: {errc}")
        except requests.exceptions.Timeout as errt:
            print(f"Timeout Error: {errt}")
        except requests.exceptions.RequestException as err:
            print(f"Request Exception: {err}")
        return None
    
    def find_abstract_element(self,html_content):
        
        soup = BeautifulSoup(html_content, 'html.parser')

    # Busca el título que incluye la palabra "Abstract"
        abstract_title = soup.find(lambda tag: tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6'] and 'Abstract' in tag.text)

        if abstract_title:
            # Encuentra el contenedor más cercano (puedes ajustar esto según la estructura de tu página)
            abstract_container = abstract_title.find_parent()

            if abstract_container:
                # Obtiene el contenido completo del contenedor
                full_content = str(abstract_container.get_text().strip())
                return full_content

        # Si no se encuentra el título "Abstract", intenta buscar por clase
        # Find any element with class containing the word "abstract"
        abstract_elements = soup.find_all(lambda tag: tag.get('class') and 'abstract' in ' '.join(tag.get('class')).lower())

        if abstract_elements:
            # Extrae el contenido de texto del primer elemento coincidente
            abstract = abstract_elements[0].get_text().strip()
            return abstract

        return None

    def chunk_text(self,text):
        #TODO
        #Cojo el texto lo divido en parrafos. Cojo todos los parrafos lo convierto a tokens. Inicializo una variable que sea la longitud en tokens que llevo para un chunk
        # Inicializo una lista con los chunks que llevo y otra donde se iran guardando los chunks finales.
        #Voy añadiendo poco a poco sumando el tamaño de cada parrafo en tokens a mi variable de longitud. A la vez si la longitud no supera un maximo voy incluyendo los parrafos en el chunk actual.
        # Si se ha llegado al chunk actual se mete en la lista de chunks y se vacia el chunk actual. La idea es cada vez que se inicie uno nuevo quedarse con el chunk anterior siempre y cuando permita sumarle el siguiente chunk de tokens para mantener el contexto.
        
        return 0
    
    def check_pdf_in_link(self,link):
        if "pdf" in link:
            return True
        else:
            return False
    def obtain_pdf(self,link,store_path,pdf_name="out.pdf"):
        #True indicates correct download
        if self.check_pdf_in_link(link):
            return self.download_pdf(link=link,store_path=store_path+pdf_name)
        else:
            try:
                pdfkit.from_url(link, pdf_name)
                return True
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 403:
                    print(f"Error 403 Forbidden: Access to {link} is restricted.")
                else:
                    print(f"HTTP Error: {e}")
                return False

    def download_pdf(self, store_path,link):
        
        response = requests.get(link)

        
        if response.status_code == 200:
            
            with open(store_path, 'wb') as archivo_local:
                archivo_local.write(response.content)
            print(f"PDF store succesfully '{store_path}'")
            return True
        else:
            print(f"Error in PDF download. Code: {response.status_code}")
            return False
        
    def store(self, store_path="text.xlsx"):
        self.dataframe.to_excel(store_path, index=False)

    def sort_links(self,links):
        return sorted(links, key=lambda link: "pdf" in link)
        
    def test(self,abstract_column_name='Abstract',title_column_name='title'):
    
        rows_with_blank_abstracts = self.dataframe[self.dataframe[abstract_column_name].isnull() | (self.dataframe[abstract_column_name] == '')]
        count=0
        for index,row in rows_with_blank_abstracts.iterrows():
            query = row['title']
            links=self.sort_links(self.search_link(query=query,sleep_interval=random.randint(a=5,b=10)))
            for link in links:
                if self.check_pdf_in_link(link):
                    #Descargar el pdf y utilizar mi modelo o otra tecnica.
                    pass
                else:
                    abstract=self.abstract_from_DOI(row["DOI"])
                    if abstract is None:
                        html_content=self.download_html(link,"./pdf/out.html")
                        if html_content is not None:
                            abstract=self.find_abstract_element(html_content=html_content)
                            if abstract is not None:
                                self.dataframe.at[index,abstract_column_name]=abstract
                                break
                            else:
                                #Hacer lo del pdf
                                pass
                        else:
                            #pasamos al siguiente link extraible
                            pass
                    else:
                        self.dataframe.at[index,abstract_column_name]=abstract
                        break
                sleep(random.uniform(0.1,3.0))
            if count%10==0:
                self.store()
            count+=1
        self.store()
                



In [59]:
test=AbstractExtractor(result)
test.test()

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ggml_cuda_set_main_device: using device 0 (NVIDIA RTX A6000) as main device


KeyboardInterrupt: 

In [15]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Authors            543 non-null    object 
 1   Researcher Ids     210 non-null    object 
 2   ORCIDs             263 non-null    object 
 3   title              543 non-null    object 
 4   Volume             279 non-null    object 
 5   Issue              164 non-null    object 
 6   DOI                543 non-null    object 
 7   Document Type      543 non-null    object 
 8   Publication Date   528 non-null    float64
 9   Abstract           455 non-null    object 
 10  ISSN               242 non-null    object 
 11  eISSN              216 non-null    object 
 12  ISBN               192 non-null    object 
 13  Pages              217 non-null    object 
 14  Publisher          543 non-null    object 
 15  Proceedings title  543 non-null    object 
 16  Keywords           80 non-

In [57]:
from googlesearch import search
import pdfkit
import pdfplumber
import requests

https://aclanthology.org/2021.semeval-1.134
https://www.academia.edu/95635312/BennettNLP_at_SemEval_2021_Task_5_Toxic_Spans_Detection_using_Stacked_Embedding_Powered_Toxic_Entity_Recognizer
https://arxiv.org/abs/2104.10100
https://paperswithcode.com/author/l-alfonso-ure-n-a-l-o-pez
https://www.researchgate.net/publication/351019396_UIT-ISE-NLP_at_SemEval-2021_Task_5_Toxic_Spans_Detection_with_BiLSTM-CRF_and_Toxic_Bert_Comment_Classification
https://paperswithcode.com/paper/lz1904-at-semeval-2021-task-5-bi-lstm-crf-for
https://github.com/sonlam1102/toxic-spans-detection-bilstm_crf
https://github.com/sonlam1102/toxic-spans-detection-bilstm_crf


KeyboardInterrupt: 

In [3]:
from googleapi import google
num_page = 3
search_results = google.search("SINAI at SemEval-2021 Task 5: Combining Embeddings in a BiLSTM-CRF model for Toxic Spans Detection", num_page)

Error accessing: http://www.google.com/search?nl=en&q=SINAI%2Bat%2BSemEval-2021%2BTask%2B5%3A%2BCombining%2BEmbeddings%2Bin%2Ba%2BBiLSTM-CRF%2Bmodel%2Bfor%2BToxic%2BSpans%2BDetection&start=0&num=10&tbs=
HTTP Error 429: Too Many Requests
Error accessing: http://www.google.com/search?nl=en&q=SINAI%2Bat%2BSemEval-2021%2BTask%2B5%3A%2BCombining%2BEmbeddings%2Bin%2Ba%2BBiLSTM-CRF%2Bmodel%2Bfor%2BToxic%2BSpans%2BDetection&start=10&num=10&tbs=
HTTP Error 429: Too Many Requests
Error accessing: http://www.google.com/search?nl=en&q=SINAI%2Bat%2BSemEval-2021%2BTask%2B5%3A%2BCombining%2BEmbeddings%2Bin%2Ba%2BBiLSTM-CRF%2Bmodel%2Bfor%2BToxic%2BSpans%2BDetection&start=20&num=10&tbs=
HTTP Error 429: Too Many Requests


In [2]:
search_results

[]

In [40]:
rows_with_blank_abstracts = result[result['Abstract'].isnull() | (result['Abstract'] == '')]

# Print the indices and titles for rows with blank abstracts
for index, row in rows_with_blank_abstracts.iterrows():
    print(f"Index: {index}, Title: {row['title']}")
    
    query = row['title']
    # Perform the Google search and print the results
    for link in search(query,num_results=5):
        print("Trying link:{link}")
        try:
            pdfkit.from_url(link, 'out.pdf')
            with pdfplumber.open('out.pdf') as pdf:
                pages = pdf.pages
                for page in pages:
                    text = page.extract_text()
                    print(text)
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 403:
                print(f"Error 403 Forbidden: Access to {link} is restricted.")
            else:
                print(f"HTTP Error: {e}")

        except Exception as e:
            print(f"Error: {e}")

Index: 100, Title: Can CBCT-Based Delta Radiomics Predict Normal Lung Toxicity during Thoracic Radiation?
Error: wkhtmltopdf reported an error:
Exit with code 1 due to network error: ContentNotFoundError

Error: wkhtmltopdf reported an error:
setCachingEnabled:  65536  bytesDownloaded
QNetworkReplyImpl: backend error: caching was enabled after some bytes had been written
Exit with code 1 due to network error: ContentNotFoundError

Error: wkhtmltopdf reported an error:
Exit with code 1 due to network error: ContentNotFoundError

Recruit researchers Join for free Login
Conference Paper
Can CBCT-Based Delta Radiomics Predict Normal Lung Toxicity during Thoracic Radiation?
November 2022 · International Journal of Radiation Oncology*Biology*Physics 114(3):e118-e119
10.1016/j.ijrobp.2022.07.934
Conference: ASTRO Annual 2022 Meeting
N. Amal Joseph Varghese Hannah Mary Thomas T A.
Jose Christian Medical College Vellore Christian Medical College Vellore Irodi
Show all 13 authors
Request full-te

KeyboardInterrupt: 

In [2]:
from pdfquery import PDFQuery


In [7]:
pdf = PDFQuery('ejemplo_local.pdf')
pdf.load()

In [8]:
pdf.tree.write('customers.xml', pretty_print = True)

In [9]:
pdf.extract("Abstract")

ValueError: not enough values to unpack (expected 3, got 2)

In [23]:
import requests

HTML content downloaded successfully and saved to out.html


In [28]:
from googlesearch import search
import pdfkit
import pdfplumber

# Specify the query
query = "Cross-Domain Toxic Spans Detection"

# Perform the Google search and print the results
for link in search(query,num_results=1):
    
    pdfkit.from_url("https://aclanthology.org/2021.semeval-1.31/", 'out.pdf')
    with pdfplumber.open('out.pdf') as pdf:
        pages = pdf.pages
        for page in pages:
            text = page.extract_text()
            print(text)

ACL Anthology
FAQ
Corrections
Submissions
Github
Search...
HamiltonDinggg at SemEval-2021 Task 5: Investigating Toxic Span Detection using RoBERTa Pre-training
Huiyang Ding, David Jurgens
Abstract
This paper presents our system submission to task 5: Toxic Spans Detection of the SemEval-2021 competition. The competition aims at detecting the spans that make a toxic span toxic. In this
paper, we demonstrate our system for detecting toxic spans, which includes expanding the toxic training set with Local Interpretable Model-Agnostic Explanations (LIME), fine-tuning RoBERTa
model for detection, and error analysis. We found that feeding the model with an expanded training set using Reddit comments of polarized-toxicity and labeling with LIME on top of logistic
regression classification could help RoBERTa more accurately learn to recognize toxic spans. We achieved a span-level F1 score of 0.6715 on the testing phase. Our quantitative and qualitative
results show that the predictions from our 

In [53]:
pdfkit.from_url("https://arxiv.org/abs/2306.09642", 'out.pdf')
with pdfplumber.open('out.pdf') as pdf:
    pages = pdf.pages
    all_text=""
    for page in pages:
        text = page.extract_text()
        all_text+=text
    # print(all_text)
    max_size = 500  # Specify the desired size limit
    segments = split_text_into_segments(all_text, max_size)

    # Print the segments
    for i, segment in enumerate(segments, start=1):
        print(f"Segment {i}:\n{segment}\n")

Segment 1:
Skip to main content
> cs > arXiv:2306.09642
Search... GO
quick links
Login
Help Pages
About
Computer Science > Computation and Language
arXiv:2306.09642 (cs)
[Submitted on 16 Jun 2023]Title:Cross-Domain Toxic Spans Detection
Authors:Stefan F. Schouten, Baran Barbarestani, Wondimagegnhue Tufa, Piek Vossen, Ilia Markov
Download PDF
Abstract:Given the dynamic nature of toxic language use, automated methods for detecting toxic spans are

Segment 2:
likely to encounter distributional shift. To explore this phenomenon, we evaluate three approaches for
detecting toxic spans under cross-domain conditions: lexicon-based, rationale extraction, and fine-tuned
language models. Our findings indicate that a simple method using off-the-shelf lexicons performs best in the
cross-domain setup. The cross-domain error analysis suggests that (1) rationale extraction methods are prone

Segment 3:
to false negatives, while (2) language models, despite performing best for the in-domain case, recal

In [50]:
def split_text_into_segments(text, max_segment_size):
    paragraphs = text.split('\n')  # Assuming paragraphs are separated by newline characters
    segments = []
    current_segment = ""

    for paragraph in paragraphs:
        if len(current_segment) + len(paragraph) <= max_segment_size:
            current_segment += paragraph + '\n'
        else:
            # If adding the current paragraph exceeds the size limit, start a new segment
            segments.append(current_segment.strip())  # Remove trailing newline
            current_segment = paragraph + '\n'

    # Add the last segment
    segments.append(current_segment.strip())

    return segments

In [6]:
import pdfkit
pdfkit.from_url('https://aclanthology.org/2021.semeval-1.6.pdf', 'out.pdf')

OSError: wkhtmltopdf exited with non-zero code 1. error:
Exit with code 1, due to unknown error.


In [7]:
import requests
def descargar_pdf(desde_url, guardar_como):
    # Realizar una solicitud GET al enlace del PDF
    response = requests.get(desde_url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Guardar el contenido del PDF en un archivo local
        with open(guardar_como, 'wb') as archivo_local:
            archivo_local.write(response.content)
        print(f"El archivo PDF se ha descargado exitosamente como '{guardar_como}'")
    else:
        print(f"Error al descargar el PDF. Código de estado: {response.status_code}")
url_del_pdf = "https://aclanthology.org/2021.semeval-1.6.pdf"
nombre_local_del_pdf = "ejemplo_local.pdf"

descargar_pdf(url_del_pdf, nombre_local_del_pdf)

El archivo PDF se ha descargado exitosamente como 'ejemplo_local.pdf'


In [3]:
pdfkit.from_url('https://arxiv.org/pdf/2306.09642.pdf', 'out1.pdf')


NameError: name 'pdfkit' is not defined

In [7]:
import fitz # imports the pymupdf library
doc = fitz.open("out.pdf") # open a document
for page in doc: # iterate the document pages
  text = page.get_text() # get plain text encoded as UTF-8
  print(text)

RESEARCH-ARTICLE
AI Principles in Identifying Toxicity in Online Conversation: Keynote at the Third
Workshop on Fairness, Accountability, Transparency, Ethics and Society on the Web
Author:  
Authors Info & Claims
WWW '21: Companion Proceedings of the Web Conference 2021 • April 2021 • Pages 237 • https://doi.org/10.1145/3442442.3452307
Published: 03 June 2021 Publication History
Lucy Vassermann
Jigsaw’s Perspective API aims to protect voices in online conversation by developing and serving machine learning models that
identify toxicity text. This talk will share how the team behind Perspective thinks about the issues of Fairness, Accountability,
Transparency, Ethics and Society through the lens of Google’s AI Principles. For the Perspective team, building technology that
is fair and ethical is a continuous, ongoing effort. The talk will cover concrete strategies the Perspective team has already used
to mitigate bias in ML models as well as new strategies currently being explored. Fina

In [44]:
import pdfplumber

with pdfplumber.open('out.pdf') as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text()
        print(text)

Skip to main content
> cs > arXiv:2306.09642
Search... GO
quick links
Login
Help Pages
About
Computer Science > Computation and Language
arXiv:2306.09642 (cs)
[Submitted on 16 Jun 2023]
Title:Cross-Domain Toxic Spans Detection
Authors:Stefan F. Schouten, Baran Barbarestani, Wondimagegnhue Tufa, Piek Vossen, Ilia Markov
Download PDF
Abstract:Given the dynamic nature of toxic language use, automated methods for detecting toxic spans are
likely to encounter distributional shift. To explore this phenomenon, we evaluate three approaches for
detecting toxic spans under cross-domain conditions: lexicon-based, rationale extraction, and fine-tuned
language models. Our findings indicate that a simple method using off-the-shelf lexicons performs best in the
cross-domain setup. The cross-domain error analysis suggests that (1) rationale extraction methods are prone
to false negatives, while (2) language models, despite performing best for the in-domain case, recall fewer
explicitly toxic words tha

In [31]:
import pdfplumber

with pdfplumber.open('out.pdf') as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text()
        print(text)

ACL Anthology
FAQ
Corrections
Submissions
Github
Search...
HamiltonDinggg at SemEval-2021 Task 5: Investigating Toxic Span Detection using RoBERTa Pre-training
Huiyang Ding, David Jurgens
Abstract
This paper presents our system submission to task 5: Toxic Spans Detection of the SemEval-2021 competition. The competition aims at detecting the spans that make a toxic span toxic. In this
paper, we demonstrate our system for detecting toxic spans, which includes expanding the toxic training set with Local Interpretable Model-Agnostic Explanations (LIME), fine-tuning RoBERTa
model for detection, and error analysis. We found that feeding the model with an expanded training set using Reddit comments of polarized-toxicity and labeling with LIME on top of logistic
regression classification could help RoBERTa more accurately learn to recognize toxic spans. We achieved a span-level F1 score of 0.6715 on the testing phase. Our quantitative and qualitative
results show that the predictions from our 

In [2]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral", gpu_layers=100,max_new_tokens = 4096,
                                           context_length = 4096)

# print(llm("AI is going to"))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ggml_cuda_set_main_device: using device 0 (NVIDIA RTX A6000) as main device


In [5]:
import pdfplumber


with pdfplumber.open('out1.pdf') as pdf:
    pages = pdf.pages
    all_text=""
    for page in pages:
        page_text = page.extract_text()
        # print(page_text)
#         text = f"""
# <s>[INST] You are an expert reader assistant. Extract the text were is written the abstract. If there is not an abstract, output "No abstract provided".
# Text: {page_text}
# Abstract:[/INST]</s>
# """
        text = f"""
<s>[INST] You are an expert reader assistant. Your task is to extract the abstract of a research from the given text, and you must provide the abstract exactly as it is formatted. If the provided text does not contain an abstract, the output format must be "No abstract provided."
Text: {page_text}
Abstract:[/INST]</s>
"""
        output=llm(text,temperature=1.0,seed=42)
        if "No abstract provided" in output:
            print("--")
            print(output)
            pass
        else:
            print(output)
            break
        print("__")
    # print(all_text)
    # text=f"""
    # <s>[INST] You are an expert reader assistant.Your task is to extract the abstract from the given information, only output the abstract as it is wthout changes. Use the following scheme:
    #  Abstract:. The text is the following: {all_text}[/INST]</s>
    # """
    # print(llm(text))

--
This abstract appears to be incomplete, please provide the rest of the abstract below or in the formatted manner that is mentioned at the end of the text.
If the provided text does not contain an abstract, the output format must be "No abstract provided."
__
Given the dynamic nature of toxic language use, automated methods for detecting toxic spans are likely to encounter distributional shift. To explore this phenomenon, we evaluate three approaches for detecting toxic spans under cross-domain conditions: lexicon-based, rationale extraction, and fine-tuned language models. Our findings indicate that a simple method using off-the-shelf lexicons performs best in the cross-domain setup. The cross-domain error analysis suggests that (1) rationale extraction methods are prone to false negatives, while (2) language models, despite performing best for the in-domain case, recall fewer explicitly toxic words than lexicons and are prone to certain types of false positives.


In [2]:
from vllm import LLM,SamplingParams

  # Sample prompts.
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1")  # Create an LLM.
# outputs = llm.generate(prompts) 

INFO 12-12 19:44:47 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.1', tokenizer='mistralai/Mistral-7B-Instruct-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


AssertionError: tensor model parallel group is already initialized

In [ ]:
sampling_params = SamplingParams(max_tokens=4096, # set it same as max_seq_length in SFT Trainer
                  skip_special_tokens=True)

In [3]:
prompts=["<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]","Hi"]
output = llm.generate(prompts,sampling_params)
output[0].outputs[0].text

Processed prompts: 100%|██████████| 2/2 [00:06<00:00,  3.33s/it]


' As the Bard once said, "What\'s past is prologue". Well, in this case, what I have is a recipe from the past. Here\'s an old English Mayonnaise recipe that features ingredients commonly used during the time of Shakespeare:\n\nIngredients:\n\n* 1 egg\n* 1/4 cup oil (such as olive or canola)\n* 1/4 cup vinegar (such as apple cider or white wine)\n* 1/2 teaspoon Dijon mustard\n* Salt and pepper, to taste\n\nInstructions:\n\n1. Crack the egg into a bowl and whisk it until it is frothy.\n2. Slowly pour the oil into the egg, whisking constantly. You may want to start by pouring a few drops of oil at a time to avoid separating the egg.\n3. Continue to whisk until the mixture thickens and combines.\n4. Add the vinegar, Dijon mustard, salt, and pepper to the mixture and stir until well combined.\n5. Cover the bowl with a cloth and let it sit at room temperature for 2-3 hours, whisking occasionally to ensure the mixture remains smooth.\n\nServe your old English Mayonnaise with your favourite s

In [ ]:
import pdfplumber
with pdfplumber.open('out.pdf') as pdf:
    pages = pdf.pages
    all_text=""
    for page in pages:
        page_text = page.extract_text()
        # print(page_text)
#         text = f"""
# <s>[INST] You are an expert reader assistant. Extract the text were is written the abstract. If there is not an abstract, output "No abstract provided".
# Text: {page_text}
# Abstract:[/INST]</s>
# """
        text = f"""
<s>[INST] You are an expert reader assistant. Your task is to extract the abstract of a research from the given text, and you must provide the abstract exactly as it is formatted. If the provided text does not contain an abstract, the output format must be "No abstract provided."
Text: {page_text}
Abstract:[/INST]</s>
"""
        output = llm.generate([text],sampling_params)
        output[0].outputs[0].text
        

In [4]:
output

[RequestOutput(request_id=2, prompt="<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]", prompt_token_ids=[1, 1, 733, 16289, 28793, 1824, 349, 574, 16020, 2076, 2487, 28804, 733, 28748, 16289, 28793, 6824, 28725, 315, 28742, 28719, 3448, 10473, 298, 264, 1179, 11322, 19961, 302, 6138, 23598, 18342, 28723, 661, 13633, 776, 272, 1103, 3558, 302, 686, 16944, 15637, 423, 298, 5681, 315, 28742, 28719, 13198, 582, 297, 272, 6132, 28808, 2, 28705, 733, 16289, 28793, 2378, 368, 506, 993, 7136, 864, 21116, 28804, 733, 28748, 16289, 28793], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' As the Bard once said, "What\'s past is prologue". Well, in this case, what I have is a recipe from the past. Here\'s an old English Mayonnaise recipe that features ingredients commonly used du

In [214]:
from crossref.restful import Works

In [215]:
works = Works()

In [12]:
works.FIELDS_QUERY

('affiliation',
 'author',
 'bibliographic',
 'chair',
 'container_title',
 'contributor',
 'editor',
 'event_acronym',
 'event_location',
 'event_name',
 'event_sponsor',
 'event_theme',
 'funder_name',
 'publisher_location',
 'publisher_name',
 'translator')

In [213]:
from habanero import Crossref

cr = Crossref()

result = cr.works(query = 'Alejandro Mosquera at DETOXIS 2021: Deep Learning Approaches to Toxicity Detection in Spanish Social Media Texts')

print(result['message']['items'][0]['DOI'])

10.5220/0009351903230330


In [13]:
print(result['message']['items'][0])

{'indexed': {'date-parts': [[2023, 12, 12]], 'date-time': '2023-12-12T22:27:48Z', 'timestamp': 1702420068981}, 'reference-count': 257, 'publisher': 'Informa UK Limited', 'issue': '1', 'content-domain': {'domain': [], 'crossmark-restriction': False}, 'short-container-title': ['Journal of Environmental Science and Health, Part C'], 'published-print': {'date-parts': [[2006, 7]]}, 'DOI': '10.1080/10590500600614303', 'type': 'journal-article', 'created': {'date-parts': [[2006, 5, 12]], 'date-time': '2006-05-12T20:12:18Z', 'timestamp': 1147464738000}, 'page': '103-154', 'source': 'Crossref', 'is-referenced-by-count': 314, 'title': ['An Evaluation of the Biological and Toxicological Properties of<i>Aloe Barbadensis</i>(Miller), Aloe Vera'], 'prefix': '10.1080', 'volume': '24', 'author': [{'given': 'MARY D.', 'family': 'BOUDREAU', 'sequence': 'first', 'affiliation': []}, {'given': 'FREDERICK A.', 'family': 'BELAND', 'sequence': 'additional', 'affiliation': []}], 'member': '301', 'reference': [

In [216]:
element=works.doi('10.18653/V1/2021.SEMEVAL-1.135 ')


In [226]:
element["abstract"]

KeyError: 'abstract'

In [1]:
from scholarly import scholarly
import requests
from bs4 import BeautifulSoup

In [6]:
from scholarly import scholarly

# Retrieve the author's data, fill-in, and print
# Get an iterator for the author results
search_query = scholarly.search_author('Steven A Cholewiak')
# Retrieve the first result from the iterator
first_author_result = next(search_query)
scholarly.pprint(first_author_result)

# Retrieve all the details for the author
author = scholarly.fill(first_author_result )
scholarly.pprint(author)

# Take a closer look at the first publication
first_publication = author['publications'][0]
first_publication_filled = scholarly.fill(first_publication)
scholarly.pprint(first_publication_filled)

# Print the titles of the author's publications
publication_titles = [pub['bib']['title'] for pub in author['publications']]
print(publication_titles)

# Which papers cited that publication?
citations = [citation['bib']['title'] for citation in scholarly.citedby(first_publication_filled)]
print(citations)

b"{'affiliation': 'Applied Vision Scientist at Google LLC',\n 'citedby': 558,\n 'email_domain': '@google.com',\n 'filled': False,\n 'interests': ['Depth Cues',\n               '3D Shape',\n               'Shape from Texture & Shading',\n               'Naive Physics',\n               'Haptics'],\n 'name': 'Steven A. Cholewiak, PhD',\n 'scholar_id': '4bahYMkAAAAJ',\n 'source': 'SEARCH_AUTHOR_SNIPPETS',\n 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=4bahYMkAAAAJ'}"
b'{\'affiliation\': \'Applied Vision Scientist at Google LLC\',\n \'citedby\': 558,\n \'citedby5y\': 389,\n \'cites_per_year\': {2008: 3,\n                    2009: 5,\n                    2010: 8,\n                    2011: 9,\n                    2012: 22,\n                    2013: 17,\n                    2014: 16,\n                    2015: 18,\n                    2016: 35,\n                    2017: 32,\n                    2018: 29,\n                    2019: 63,\n                    2

In [7]:
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
scholarly.pprint(next(search_query))

StopIteration: 

In [24]:
from scholarly import ProxyGenerator
pg = ProxyGenerator()
success = pg.SingleProxy(https = "5.161.212.254:3128")
scholarly.use_proxy(pg)

TypeError: 'NoneType' object is not subscriptable

In [25]:
from scholarly import ProxyGenerator

# Set up a ProxyGenerator object to use free proxies
# This needs to be done only once per session
pg = ProxyGenerator()
success = pg.ScraperAPI("c96ecab567b075d19d4b098d79e36860")
scholarly.use_proxy(pg)

# Now search Google Scholar from behind a proxy
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
scholarly.pprint(next(search_query))

MaxTriesExceededException: Cannot Fetch from Google Scholar.

In [5]:
# Now search Google Scholar from behind a proxy
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
scholarly.pprint(next(search_query))

MaxTriesExceededException: Cannot Fetch from Google Scholar.

In [4]:
import requests

In [5]:
def download_html(url, save_path):
    try:
        # Send a GET request to the URL
        session = requests.Session()
        response = session.get(url)
        # response = requests.get(url,headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Save the HTML content to a file
        with open(save_path, 'w', encoding='utf-8') as file:
            file.write(response.text)
        
        print(f"HTML content downloaded successfully and saved to {save_path}")
    
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception: {err}")

In [10]:
download_html(url="https://aclanthology.org/2021.semeval-1.31/",save_path="out.html")

HTML content downloaded successfully and saved to out.html


In [202]:
download_html(url="https://dl.acm.org/doi/fullHtml/10.1145/3442442.3452307",save_path="out.html")

HTML content downloaded successfully and saved to out.html


In [6]:
download_html(url="https://ieeexplore.ieee.org/document/7382991",save_path="out.html")

HTTP Error: 418 Client Error: Unknown Code for url: https://ieeexplore.ieee.org/document/7382991


In [203]:
from bs4 import BeautifulSoup

def find_abstract_element(html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find any element with class containing the word "abstract"
    abstract_elements = soup.find_all(lambda tag: tag.get('class') and 'abstract' in ' '.join(tag.get('class')).lower())

    if abstract_elements:
        # Extract the text content of the first matching element
        abstract = abstract_elements[0].get_text().strip()
        return abstract

    return None

# Example usage
html_path = "/home/d4k/Documents/guillermo/doctorado/systematic review/out.html"
abstract = find_abstract_element(html_path)
print(abstract)

Jigsaw's Perspective API aims to protect voices in online conversation by developing and serving machine learning models that identify toxicity text. This talk will share how the team behind Perspective thinks about the issues of Fairness, Accountability, Transparency, Ethics and Society through the lens of Google's AI Principles. For the Perspective team, building technology that is fair and ethical is a continuous, ongoing effort. The talk will cover concrete strategies the Perspective team has already used to mitigate bias in ML models as well as new strategies currently being explored. Finally, with examples of how Perspective is being used in the real world, the talk will show how machine learning, combined with thoughtful human moderation and participation, can help improve online conversations.


In [237]:
from bs4 import BeautifulSoup

def find_abstract_element(html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Busca el título que incluye la palabra "Abstract"
    abstract_title = soup.find(lambda tag: tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6'] and 'Abstract' in tag.text)

    if abstract_title:
        # Encuentra el contenedor más cercano (puedes ajustar esto según la estructura de tu página)
        abstract_container = abstract_title.find_parent()

        if abstract_container:
            # Obtiene el contenido completo del contenedor
            full_content = str(abstract_container.get_text().strip())
            return full_content

    # Si no se encuentra el título "Abstract", intenta buscar por clase
    # Find any element with class containing the word "abstract"
    abstract_elements = soup.find_all(lambda tag: tag.get('class') and 'abstract' in ' '.join(tag.get('class')).lower())

    if abstract_elements:
        # Extrae el contenido de texto del primer elemento coincidente
        abstract = abstract_elements[0].get_text().strip()
        return abstract
    
    abstract_elements = soup.find(lambda tag: tag.get('id') and 'abstract' in tag['id'].lower())

    if abstract_elements:
        # Extract text content from the matching element
        abstract = abstract_elements.get_text().strip()
        return abstract

    return None

# Example usage
html_path = "/home/d4k/Documents/guillermo/doctorado/systematic review/out.html"
abstract = find_abstract_element(html_path)
print(abstract)

AbstractThe upsurge of prolific blogging and microblogging platforms enabled the abusers to spread negativity and threats greater than ever. Detecting the toxic portions substantially aids to moderate or exclude the abusive parts for maintaining sound online platforms. This paper describes our participation in the SemEval 2021 toxic span detection task. The task requires detecting spans that convey toxic remarks from the given text. We explore an ensemble of sequence labeling models including the BiLSTM-CRF, spaCy NER model with custom toxic tags, and fine-tuned BERT model to identify the toxic spans. Finally, a majority voting ensemble method is used to determine the unified toxic spans. Experimental results depict the competitive performance of our model among the participants.


In [199]:
# Sample list of links
links = [
     "https://example.com/pdf-page-1.pdf",
    "https://example.com/relevant-page",
    "https://example.com/another-relevant-page",
    "https://example.com/pdf-page-2.pdf",
    "https://example.com/another-relevant-pdf-page.pdf",
]

# Sort the links based on the presence of "pdf" in the link
sorted_links = sorted(links, key=lambda link: "pdf" not in link)


# Print the sorted links
for link in sorted_links:
    print(link)

https://example.com/pdf-page-1.pdf
https://example.com/pdf-page-2.pdf
https://example.com/another-relevant-pdf-page.pdf
https://example.com/relevant-page
https://example.com/another-relevant-page


In [14]:
def custom_sort(links):
    # Define the priority domains
    priority_domains = ['arxiv.org', 'dl.acm.org']

    # Check if the link contains any of the priority domains
    for domain in priority_domains:
        if domain in links:
            return (0, links)  # Put links with priority domains first

    # For links without priority domains, maintain their original order
    return (1, links)

# Example list of links
links = [
    'https://example.com/789',
    'https://anotherexample.org/101',
    'https://dl.acm.org/456',
    'https://arxiv.org/123',
    
]

# Sort the links using the custom sorting function
sorted_links = sorted(links, key=custom_sort)

# Print the sorted links
print(sorted_links)

['https://arxiv.org/123', 'https://dl.acm.org/456', 'https://anotherexample.org/101', 'https://example.com/789']


In [17]:
from transformers import AutoTokenizer
autotokenizer= AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [29]:
large_text="""In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
In a world where technology continues to advance at an unprecedented pace, the impact on various aspects of our lives is undeniable. From the way we communicate to the manner in which we conduct business, the role of technology is ever-expanding. The digital landscape is constantly evolving, bringing forth new opportunities and challenges.

Artificial intelligence, machine learning, and data analytics have become integral components of decision-making processes in fields ranging from healthcare to finance. The convergence of these technologies has given rise to innovations that were once thought to be the realm of science fiction. Autonomous vehicles, smart cities, and advanced robotics are no longer distant concepts but rather tangible realities.

As we navigate this era of digital transformation, questions surrounding ethics, privacy, and security loom large. The responsible use of technology is a pressing concern that requires careful consideration. Striking a balance between innovation and ethical considerations is essential to ensure that the benefits of technology are realized without compromising fundamental values.

In the realm of communication, social media platforms have revolutionized the way we connect with others. The exchange of information happens in real-time, transcending geographical boundaries. However, concerns about the impact of social media on mental health, the spread of misinformation, and the erosion of privacy have emerged as significant challenges.

The world of work is undergoing a paradigm shift, with remote and flexible work arrangements becoming more prevalent. Collaboration tools, video conferencing, and virtual workspaces have become indispensable in facilitating seamless communication and collaboration across distances. The traditional office setup is evolving, raising questions about the future of work and the nature of organizational structures.

The advent of the Internet of Things (IoT) has connected devices in ways previously unimaginable. Smart homes, wearable technology, and interconnected ecosystems are reshaping our daily lives. The proliferation of data generated by these connected devices presents opportunities for insights and efficiencies but also raises concerns about data security and privacy.

In the realm of healthcare, technology is playing a transformative role in diagnostics, treatment, and patient care. Telemedicine, wearable health monitors, and predictive analytics are enhancing healthcare delivery. However, the ethical implications of genetic testing, the responsible use of patient data, and the digital divide in access to healthcare services are subjects of ongoing debate.

Education is undergoing a digital revolution, with online learning platforms, virtual classrooms, and e-learning resources becoming commonplace. The accessibility of educational content has increased, but challenges such as the digital divide, the quality of online education, and the role of technology in fostering critical thinking skills remain areas of active exploration.

As we stand at the crossroads of technological advancement, the choices we make today will shape the trajectory of our future. It is imperative to approach these developments with a thoughtful and ethical mindset, ensuring that technology serves as a force for good. Navigating the complexities of this digital age requires collaboration, innovation, and a commitment to upholding the values that define our humanity.
"""

In [32]:
len(autotokenizer.tokenize(text=large_text,max_length=4096,truncation=False))

6640

In [43]:
import re

In [49]:
html_text="<jats:p>How two people speak to one another depends heavily on the nature of their relationship. For example, the same phrase said to a friend in jest may be offensive to a stranger. In this paper, we apply this simple observation to study toxic comments in online social networks. We curate a collection of 6.7K tweets containing potentially toxic terms from users with different relationship types, as determined by the nature of their follower-friend connection. We find that such tweets between users with no connection are nearly three times as likely to be toxic as those between users who are mutual friends, and that taking into account this relationship type improves toxicity detection methods by about 5% on average. Furthermore, we provide a descriptive analysis of how toxic language varies by relationship type, finding for example that mildly offensive terms are used to express hostility more commonly between users with no social connection than users who are mutual friends.</jats:p>"
re.search(r'<jats:p>(.*?)<\/jats:p>', html_text, re.DOTALL).group(1)

'How two people speak to one another depends heavily on the nature of their relationship. For example, the same phrase said to a friend in jest may be offensive to a stranger. In this paper, we apply this simple observation to study toxic comments in online social networks. We curate a collection of 6.7K tweets containing potentially toxic terms from users with different relationship types, as determined by the nature of their follower-friend connection. We find that such tweets between users with no connection are nearly three times as likely to be toxic as those between users who are mutual friends, and that taking into account this relationship type improves toxicity detection methods by about 5% on average. Furthermore, we provide a descriptive analysis of how toxic language varies by relationship type, finding for example that mildly offensive terms are used to express hostility more commonly between users with no social connection than users who are mutual friends.'

In [238]:
from fuzzywuzzy import fuzz
def similarity_score(query, result):
    score = fuzz.ratio(query.lower(), result.lower())
    if score<50 and result in query:
        score=51
    return score

# Example usage
query = 'CSECU-DSG at SemEval-2021 Task 5: Leveraging Ensemble of Sequence Tagging Models for Toxic Spans Detection'
result = 'Leveraging fusion of sequence tagging models for toxic spans'

similarity = similarity_score(query, result)
print(similarity)

66


In [230]:
from google_search import search
query="CSECU-DSG at SemEval-2021 Task 5: Leveraging Ensemble of Sequence Tagging Models for Toxic Spans Detection"
result=search(query,num_results=10,proxy=None,sleep_interval=5,advanced=True)


In [231]:
for element in result:
    print(f"{element.title}:{similarity_score(query, element.title)},{element.url}")

CSECU-DSG at SemEval-2021 Task 5: Leveraging ...:58,https://aclanthology.org/2021.semeval-1.135
[PDF] SemEval-2021 Task 5: Toxic Spans Detection:57,https://www.semanticscholar.org/paper/SemEval-2021-Task-5%3A-Toxic-Spans-Detection-Pavlopoulos-Sorensen/76ecdc5441e5e6c07e250c8f45b6d8367acec56f
Fareen Tasneem:15,https://paperswithcode.com/author/fareen-tasneem
Leveraging fusion of sequence tagging models for toxic spans ...:66,https://dl.acm.org/doi/10.1016/j.neucom.2022.05.049
[2105.13959] Cisco at SemEval-2021 Task 5: What's Toxic?:44,https://arxiv.org/abs/2105.13959
Abu Nowshed Chy:2,https://paperswithcode.com/search?q=author%3AAbu+Nowshed+Chy&order_by=date
No result found


In [73]:
string.title

'Interactive Auditing of Automatic Toxicity Detection Models'

In [72]:
if query in string.title:
    print(True)
else:
    print(False)

False


In [178]:
import re

def extract_abstract(text):
    # Utiliza una expresión regular para capturar el contenido del abstract
    match = re.search(r'Abstract\s*(?::)?\s*(.*?)(?=1\sIntroduction|$)', text, re.DOTALL | re.IGNORECASE)


    if match:
        abstract_content = match.group(1).strip()
        return abstract_content
    else:
        return None

In [187]:
import fitz
from multi_column import column_boxes

doc = fitz.open("/home/d4k/Documents/guillermo/doctorado/systematic review/pdf/out.pdf")
for page in doc:
    text=""
    block_result=""
    page_result=""
    bboxes = column_boxes(page, footer_margin=50, no_image_text=True)
    for rect in bboxes:
        block_text=page.get_text(clip=rect, sort=True)
        text+=block_text
        block_result_=extract_abstract(block_text)
        if block_result_ is not None:
            block_result=block_result_   
    
    page_result_=extract_abstract(text)
    if page_result_ is not None:
        page_result=page_result_
    # Verificar si alguna de las variables es None y seleccionar la que no es None
    if page_result is not None and block_result is not None:
        # Ambas variables tienen valores, compara sus longitudes
        if len(page_result) > len(block_result):
            print(f"Page_result: {page_result}")
        else:
            print(f"Block_result: {block_result}")
    elif page_result is not None:
        # Solo page_result tiene valor, imprímelo
        print(f"Page_result: {page_result}")
    elif block_result is not None:
        # Solo block_result tiene valor, imprímelo
        print(f"Block_result: {block_result}")
    else:
        # Ambas variables son None, manejar el caso según sea necesario
        print("Ambas variables son None.")
    print("-" * 80)

Block_result: The Toxic Spans Detection task of SemEval-
2021 required participants to predict the spans
of toxic posts that were responsible for the
toxic label of the posts. The task could be ad-
dressed as supervised sequence labeling, using
training data with gold toxic spans provided
by the organisers. It could also be treated as
rationale extraction, using classiﬁers trained
on potentially larger external datasets of posts
manually annotated as toxic or not, without
toxic span annotations. For the supervised se-
quence labeling approach and evaluation pur-
poses, posts previously labeled as toxic were
crowd-annotated for toxic spans. Participants
submitted their predicted spans for a held-out
test set, and were scored using character-based
F1. This overview summarises the work of the
36 teams that provided system descriptions.
--------------------------------------------------------------------------------
Block_result: 
-----------------------------------------------------------

In [197]:
from abstract_extraction import AbstractExtractor
test=AbstractExtractor(None)

In [198]:
test.download_pdf(link="https://arxiv.org/pdf/2306.09642v1.pdf",store_path="/home/d4k/Documents/guillermo/doctorado/systematic review/pdf/test.pdf")

PDF store succesfully '/home/d4k/Documents/guillermo/doctorado/systematic review/pdf/test.pdf'


True